In [1]:
# Configuration: Set your roll number here
ROLL_NUMBER = "22bec021"  # Replace with your actual roll number

# Input file paths
LECTURE_TRANSCRIPT_PATH = "C:/Users/jallu/Downloads/23Aug2025 T+S(Agentic AI)/23Aug2025 T+S(Agentic AI)/Agentic AI (2025-08-23 09_02 GMT+5_30) - Transcript.docx"
STUDENT_SUMMARIES_PATH = "C:/Users/jallu/Downloads/23Aug2025 T+S(Agentic AI)/23Aug2025 T+S(Agentic AI)/Attendance + Summary in context learning.xlsx"

# Output file path
OUTPUT_FILE_PATH = f"Gradedresults_23rd_august.xlsx"


In [2]:
# Install required packages if not already installed
import subprocess
import sys

def install_package(package, import_name=None):
    """
    Install a package if not already installed.
    
    Args:
        package (str): Package name for pip install
        import_name (str): Name to use for import (if different from package name)
    """
    if import_name is None:
        # Map package names to their import names
        import_map = {
            "python-docx": "docx",
            "scikit-learn": "sklearn"
        }
        import_name = import_map.get(package, package.replace("-", "_"))
    
    try:
        __import__(import_name)
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package], 
                            stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# Install required packages
packages = [
    ("python-docx", "docx"),
    ("pandas", "pandas"),
    ("openpyxl", "openpyxl"),
    ("sentence-transformers", "sentence_transformers"),
    ("numpy", "numpy"),
    ("scikit-learn", "sklearn")
]

for package, import_name in packages:
    install_package(package, import_name)

print("All required packages installed successfully!")


c:\Users\jallu\OneDrive\Desktop\Quiz\quizenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


All required packages installed successfully!


In [3]:
# Import all required libraries
import os
import pandas as pd
import numpy as np
from docx import Document
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
import re  # <-- ADDED THIS (replaces nltk)
warnings.filterwarnings('ignore')

print("Libraries imported successfully (NLTK has been removed).")

Libraries imported successfully (NLTK has been removed).


In [4]:
def load_lecture_transcript(docx_path):
    """
    Load and extract text from a DOCX file containing the lecture transcript.
    
    Args:
        docx_path (str): Path to the DOCX file
    
    Returns:
        str: Full text content of the lecture transcript
    """
    try:
        if not os.path.exists(docx_path):
            raise FileNotFoundError(f"Lecture transcript file not found: {docx_path}")
        
        doc = Document(docx_path)
        full_text = []
        
        # Extract text from all paragraphs
        for paragraph in doc.paragraphs:
            if paragraph.text.strip():  # Skip empty paragraphs
                full_text.append(paragraph.text.strip())
        
        # Join all paragraphs with newlines
        transcript = "\n".join(full_text)
        
        # Validate that transcript is not empty
        if len(transcript.strip()) < 50:
            raise ValueError("Lecture transcript appears to be empty or too short. Please check the file.")
        
        print(f"Successfully loaded lecture transcript ({len(transcript)} characters)")
        return transcript
    except Exception as e:
        print(f"Error loading lecture transcript: {e}")
        raise

# Load the lecture transcript
lecture_transcript = load_lecture_transcript(LECTURE_TRANSCRIPT_PATH)
print(f"\nFirst 500 characters of transcript:\n{lecture_transcript[:500]}...")


Successfully loaded lecture transcript (80986 characters)

First 500 characters of transcript:
Agentic AI (2025-08-23 09:02 GMT+5:30) - Transcript
Attendees
A M D PRADEEP, ABHAY TIWARI, ABHISHEK BUDDIGA, AMAN CHAURASIA, AMOD ADARSH, Amrita Kadam, AMRITA KADAM, ANNEM VENKATA KISHAN KUMAR REDDY IIIT Dharwad, ANUJA GUPTA, ARAVIND BASINI, ARYAN PRATIK, Ashutosh G Singh, ATLA SMARAN REDDY, AVISHAKULA JEEVAN KUMAR, BADA POOJITHA, BANDARU USHA NAGA SRI, BASWA TANUSREE REDDY, BOBBALACHINNOLLA JASWANTH REDDY, BONGU ASHISH IIIT Dharwad, BYREDDY VASUDEVA REDDY, CHAITRA V KATTIMANI IIIT Dharwad, DARS...


In [5]:
def load_student_summaries(excel_path):
    """
    Load student summaries from an Excel file.
    
    Args:
        excel_path (str): Path to the Excel file
    
    Returns:
        pd.DataFrame: DataFrame containing student information and summaries
    """
    try:
        if not os.path.exists(excel_path):
            raise FileNotFoundError(f"Student summaries file not found: {excel_path}")
        
        df = pd.read_excel(excel_path)
        
        # Validate that DataFrame is not empty
        if len(df) == 0:
            raise ValueError("Student summaries file is empty. Please check the file.")
        
        # Expected columns: Email Address, Name of the Student, Roll Number, Institute, Summary
        print(f"Loaded {len(df)} student summaries")
        print(f"\nColumns found: {list(df.columns)}")
        print(f"\nFirst few rows:")
        print(df.head())
        
        return df
    except Exception as e:
        print(f"Error loading student summaries: {e}")
        raise

# Load student summaries
student_df = load_student_summaries(STUDENT_SUMMARIES_PATH)


Loaded 123 student summaries

Columns found: ['Timestamp', 'Email Address', 'Name of the Student', 'Roll Number', 'Institute', 'Summary( upto 100words)']

First few rows:
                Timestamp                  Email Address Name of the Student  \
0 2025-08-23 10:09:12.627          cs22b1024@iiitr.ac.in      Deepak sharma    
1 2025-08-23 10:09:17.939  jyoti.24phdcs08@iiitdwd.ac.in         Jyoti Gadad   
2 2025-08-23 10:11:19.641          cs22b1003@iiitr.ac.in       Aditya Gupta    
3 2025-08-23 10:15:00.025          cs22b1007@iiitr.ac.in      Aman Chaurasia   
4 2025-08-23 10:20:59.063         23bds032@iiitdwd.ac.in      Manikesh Kumar   

  Roll Number     Institute                            Summary( upto 100words)  
0  Cs22b1024   IIIT Raichur  We studied model selection from a wide range o...  
1   24PHDCS08  IIIT Dharwad  LangChain helps in building pipelines where an...  
2  Cs22b1003   IIIT Raichur  In today’s session, we focused on model select...  
3   CS22B1007  IIIT Raic

In [6]:
# Initialize Sentence Transformer model for semantic similarity
# Using a pre-trained model that works well for semantic similarity tasks
print("Loading Sentence Transformer model...")
print("Note: This may take a few minutes on first run as the model downloads (~80MB)")
try:
    model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight and effective model
    print("Model loaded successfully!")
except Exception as e:
    print(f"Error loading model: {e}")
    print("Please ensure you have an internet connection for the first run.")
    raise


Loading Sentence Transformer model...
Note: This may take a few minutes on first run as the model downloads (~80MB)
Model loaded successfully!


In [7]:
def extract_key_points(transcript, max_sentences=50):
    """
    Extract key points from the lecture transcript by splitting into sentences
    and identifying important sections.
    
    Args:
        transcript (str): Full lecture transcript text
        max_sentences (int): Maximum number of sentences to consider
    
    Returns:
        list: List of key sentences/points from the transcript
    """
    # Split transcript into sentences
    sentences = transcript.replace('\n', ' ').split('.')
    sentences = [s.strip() for s in sentences if len(s.strip()) > 20]  # Filter very short sentences
    
    # Take a representative sample of sentences (beginning, middle, end)
    if len(sentences) > max_sentences:
        # Sample from different parts of the transcript
        step = len(sentences) // max_sentences
        key_sentences = sentences[::step][:max_sentences]
    else:
        key_sentences = sentences
    
    return key_sentences

# Extract key points from lecture transcript
key_points = extract_key_points(lecture_transcript)
print(f"Extracted {len(key_points)} key points from the lecture transcript")
print(f"\nSample key points:\n{key_points[:3]}")


Extracted 50 key points from the lecture transcript

Sample key points:
['Agentic AI (2025-08-23 09:02 GMT+5:30) - Transcript Attendees A M D PRADEEP, ABHAY TIWARI, ABHISHEK BUDDIGA, AMAN CHAURASIA, AMOD ADARSH, Amrita Kadam, AMRITA KADAM, ANNEM VENKATA KISHAN KUMAR REDDY IIIT Dharwad, ANUJA GUPTA, ARAVIND BASINI, ARYAN PRATIK, Ashutosh G Singh, ATLA SMARAN REDDY, AVISHAKULA JEEVAN KUMAR, BADA POOJITHA, BANDARU USHA NAGA SRI, BASWA TANUSREE REDDY, BOBBALACHINNOLLA JASWANTH REDDY, BONGU ASHISH IIIT Dharwad, BYREDDY VASUDEVA REDDY, CHAITRA V KATTIMANI IIIT Dharwad, DARSHAN GOWDA D S, DEBOJYOTI ROY, DEEPAK KUMAR, DEEPAK SHARMA, DESAI KARTIK AMIT, DEVA ANAND M, GAIKWAD MANISH BABAN, GANGULA SAICHARAN REDDY, GOGA JAYA SANKEERTH IIIT Dharwad, GUGULOTH JASHWANTH, HARSHAVARDHANA BABU, HEMANT PARTE, HITIK ADWANI IIIT Dharwad, Indi Jayashree M, INTURI MOKSHAGNA IIIT Dharwad, JAKKIREDDY GANESH KUMAR REDDY IIIT Dharwad, JATAVATH DEEPTHI BAI, Jyothi Gadad IIIT Dharwad, K V Jaya Harsha, KAGANA AKSHA

In [8]:
def calculate_semantic_coverage(summary, key_points, model):
    """
    Calculates a robust score for semantic coverage.
    
    Compares each key point against each sentence in the summary
    to find the best possible match.
    
    Args:
        summary (str): Student's summary text
        key_points (list): List of key points from the lecture
        model: SentenceTransformer model
    
    Returns:
        float: A score (0-1) representing the average "best match" 
               similarity for all key points.
    """
    if not summary or not key_points or len(summary.strip()) < 10:
        return 0.0

    # 1. Split summary into sentences and get embeddings
    summary_sentences = split_into_sentences(summary)
    if not summary_sentences:
        return 0.0
    
    summary_embeddings = model.encode(summary_sentences, 
                                      convert_to_numpy=True, 
                                      show_progress_bar=False)

    # 2. Get embeddings for all key points
    key_point_embeddings = model.encode(key_points, 
                                        convert_to_numpy=True, 
                                        show_progress_bar=False)

    best_match_scores = []

    # 3. For each key point, find its best match in the summary sentences
    for key_point_emb in key_point_embeddings:
        # Calculate similarity between this key point and ALL summary sentences
        similarities = cosine_similarity([key_point_emb], summary_embeddings)[0]
        
        # Find the highest similarity (the best match for this key point)
        best_match = np.max(similarities)
        best_match_scores.append(best_match)
    
    # 4. The final score is the average of all "best match" scores
    if not best_match_scores:
        return 0.0
        
    coverage_score = np.mean(best_match_scores)
    
    return float(coverage_score)


def calculate_clarity_score(summary):
    """
    Calculate clarity and coherence score based on text characteristics.
    (This function is unchanged)
    
    Args:
        summary (str): Student's summary text
    
    Returns:
        float: Clarity score between 0 and 1
    """
    if not summary or len(summary.strip()) < 10:
        return 0.0
    
    # Basic heuristics for clarity
    word_count = len(summary.split())
    sentence_count = len(split_into_sentences(summary)) # Use new sentence splitter
    
    # Check for reasonable sentence length (not too short, not too long)
    avg_sentence_length = word_count / max(sentence_count, 1)
    
    # Score based on:
    # - Has reasonable length (not too short, not too verbose)
    # - Has multiple sentences (shows structure)
    # - Average sentence length is reasonable (10-25 words is good)
    
    length_score = min(1.0, word_count / 100)  # Prefer summaries with at least 100 words
    structure_score = min(1.0, sentence_count / 5)  # Prefer at least 5 sentences
    sentence_quality = 1.0 if 10 <= avg_sentence_length <= 30 else 0.7
    
    # Combined clarity score
    clarity = (length_score * 0.3 + structure_score * 0.3 + sentence_quality * 0.4)
    
    return min(clarity, 1.0)


def check_grammar_basic(summary):
    """
    Basic grammar check using simple heuristics.
    (This function is unchanged, but uses the new sentence splitter)
    
    Args:
        summary (str): Student's summary text
    
    Returns:
        float: Grammar score between 0 and 1
    """
    if not summary or len(summary.strip()) < 10:
        return 0.0
    
    # Basic checks:
    # - Proper capitalization at sentence start
    # - No excessive repeated characters
    
    sentences = split_into_sentences(summary) # Use new sentence splitter
    if not sentences:
        return 0.0
    
    proper_caps = sum(1 for s in sentences if s and s[0].isupper()) / len(sentences)
    
    # Check for excessive repetition (e.g., "aaaa")
    has_repetition = any(len(set(word)) == 1 and len(word) > 3 for word in summary.split())
    repetition_penalty = 0.0 if has_repetition else 1.0
    
    grammar_score = (proper_caps * 0.7 + repetition_penalty * 0.3)
    
    return grammar_score

print("Evaluation functions defined successfully!")

Evaluation functions defined successfully!


In [9]:
import re

def split_into_sentences(text):
    """
    Splits a block of text into a list of sentences using regex.
    This function NO LONGER uses NLTK.
    """
    if not text or len(text.strip()) == 0:
        return []
    
    # Split using regex: split after a period, ?, or ! followed by a space or newline
    sentences = re.split(r'(?<=[.!?])[\s\n]+', text)
    
    # Clean up and filter out very short sentences
    cleaned_sentences = [s.strip() for s in sentences if s and len(s.strip()) > 10]
    return cleaned_sentences

print("Sentence splitting helper function defined (using Regex, not NLTK)!")

Sentence splitting helper function defined (using Regex, not NLTK)!


In [10]:
# THIS IS THE REPLACEMENT FOR YOUR "EVALUATION FUNCTION" CELL (CELL 6)

def calculate_semantic_coverage(summary, key_points, model):
    """
    Calculates a robust AND SCALED score for semantic coverage.
    
    The final "raw" score is scaled to fit a 0-1 grading range.
    
    Args:
        summary (str): Student's summary text
        key_points (list): List of key points from the lecture
        model: SentenceTransformer model
    
    Returns:
        float: A SCALED score (0-1) representing coverage.
    """
    if not summary or not key_points or len(summary.strip()) < 10:
        return 0.0

    # 1. Split summary into sentences and get embeddings
    summary_sentences = split_into_sentences(summary) # This function is in another cell
    if not summary_sentences:
        return 0.0
    
    summary_embeddings = model.encode(summary_sentences, 
                                      convert_to_numpy=True, 
                                      show_progress_bar=False)

    # 2. Get embeddings for all key points
    key_point_embeddings = model.encode(key_points, 
                                        convert_to_numpy=True, 
                                        show_progress_bar=False)

    best_match_scores = []

    # 3. For each key point, find its best match in the summary sentences
    for key_point_emb in key_point_embeddings:
        similarities = cosine_similarity([key_point_emb], summary_embeddings)[0]
        best_match = np.max(similarities)
        best_match_scores.append(best_match)
    
    # 4. The raw score is the average of all "best match" scores
    if not best_match_scores:
        return 0.0
        
    raw_coverage_score = np.mean(best_match_scores)
    
    # --- 5. NEW, MORE GENEROUS SCALING ---
    # The raw scores are very low. We will map a much
    # lower range to the 0.0 to 1.0 grade.
    
    # *** YOU CAN TUNE THESE TWO NUMBERS ***
    # Make them lower to give higher grades.
    # Make them higher to give lower grades.
    MIN_SIMILARITY_THRESHOLD = 0.1   # The raw score that maps to 0.0
    MAX_SIMILARITY_THRESHOLD = 0.35  # The raw score that maps to 1.0
    
    if raw_coverage_score < MIN_SIMILARITY_THRESHOLD:
        scaled_score = 0.0
    elif raw_coverage_score > MAX_SIMILARITY_THRESHOLD:
        scaled_score = 1.0
    else:
        # Linear scaling
        scaled_score = (raw_coverage_score - MIN_SIMILARITY_THRESHOLD) / (MAX_SIMILARITY_THRESHOLD - MIN_SIMILARITY_THRESHOLD)
        
    scaled_score = np.clip(scaled_score, 0, 1)

    return float(scaled_score)


# --- THE FUNCTIONS BELOW ARE UNCHANGED ---
# (They must be in the same cell)

def calculate_clarity_score(summary):
    if not summary or len(summary.strip()) < 10:
        return 0.0
    word_count = len(summary.split())
    sentence_count = len(split_into_sentences(summary)) 
    avg_sentence_length = word_count / max(sentence_count, 1)
    length_score = min(1.0, word_count / 100)
    structure_score = min(1.0, sentence_count / 5)
    sentence_quality = 1.0 if 10 <= avg_sentence_length <= 30 else 0.7
    clarity = (length_score * 0.3 + structure_score * 0.3 + sentence_quality * 0.4)
    return min(clarity, 1.0)


def check_grammar_basic(summary):
    if not summary or len(summary.strip()) < 10:
        return 0.0
    sentences = split_into_sentences(summary) 
    if not sentences:
        return 0.0
    proper_caps = sum(1 for s in sentences if s and s[0].isupper()) / len(sentences)
    has_repetition = any(len(set(word)) == 1 and len(word) > 3 for word in summary.split())
    repetition_penalty = 0.0 if has_repetition else 1.0
    grammar_score = (proper_caps * 0.7 + repetition_penalty * 0.3)
    return grammar_score

print("Evaluation functions defined successfully (WITH NEW, LOWER SCALING)!")

Evaluation functions defined successfully (WITH NEW, LOWER SCALING)!


In [11]:
def generate_explanation(score, semantic_coverage, clarity, completeness, grammar, summary, word_count):
    """
    Generate a 2-3 sentence explanation justifying the score.
    (UPDATED to use new 'semantic_coverage' score)
    
    Args:
        score (float): Final score out of 10
        semantic_coverage (float): The new combined coverage/similarity score
        clarity (float): Clarity score
        completeness (float): Completeness score
        grammar (float): Grammar score
        summary (str): The summary text
        word_count (int): Word count of summary
    
    Returns:
        str: Explanation text
    """
    strengths = []
    weaknesses = []
    
    # Identify strengths
    if semantic_coverage > 0.6:
        strengths.append("covers most key lecture points well")
    elif semantic_coverage > 0.4:
        strengths.append("covers some key lecture points")

    if clarity > 0.7:
        strengths.append("demonstrates clear and coherent writing")
    
    if grammar > 0.8:
        strengths.append("shows good grammatical structure")
    
    # Identify weaknesses
    if semantic_coverage < 0.3:
        weaknesses.append("omits many key lecture points and shows limited alignment with lecture content")
    elif semantic_coverage < 0.4:
        weaknesses.append("misses some important lecture points")
    
    if clarity < 0.5:
        weaknesses.append("lacks clarity and coherent structure")
    
    if word_count < 50:
        weaknesses.append("is too brief and lacks detail")
    
    if grammar < 0.6:
        weaknesses.append("contains grammatical issues")
    
    # Construct explanation
    explanation_parts = []
    
    if strengths:
        explanation_parts.append(f"The summary {', '.join(strengths[:2])}.")
    
    if weaknesses:
        explanation_parts.append(f"However, it {', '.join(weaknesses[:2])}.")
    
    if not explanation_parts:
        if score >= 7:
            explanation_parts.append("The summary provides a solid overview of the lecture with good coverage and clarity.")
        elif score >= 4:
            explanation_parts.append("The summary demonstrates moderate understanding but could benefit from more comprehensive coverage.")
        else:
            explanation_parts.append("The summary requires significant improvement in coverage, clarity, and alignment with lecture content.")
    
    # Add score context
    if score >= 8:
        explanation_parts.append("Overall, this represents a strong summary that effectively captures the lecture's main points.")
    elif score >= 6:
        explanation_parts.append("The summary meets basic requirements but has room for improvement in depth and completeness.")
    elif score >= 4:
        explanation_parts.append("The summary shows partial understanding but lacks sufficient detail and coverage.")
    else:
        explanation_parts.append("The summary needs substantial revision to adequately represent the lecture content.")
    
    return " ".join(explanation_parts)


def evaluate_summary(summary, lecture_transcript, key_points, model):
    """
    Comprehensive evaluation of a student summary against the lecture transcript.
    (UPDATED to use new 'calculate_semantic_coverage' function and new weights)
    
    Args:
        summary (str): Student's summary text
        lecture_transcript (str): Full lecture transcript (unused, but kept for compatibility)
        key_points (list): List of key points from the lecture
        model: SentenceTransformer model for embeddings
    
    Returns:
        tuple: (score out of 10, explanation string)
    """
    if not summary or len(summary.strip()) < 10:
        return 0.0, "Summary is too short or empty. No meaningful content provided."
    
    # Calculate individual component scores (all between 0 and 1)
    
    # 1. NEW: Semantic Coverage (Replaces old 'similarity' and 'coverage')
    # This is now the main semantic score, comparing summary sentences to key points.
    semantic_coverage = calculate_semantic_coverage(summary, key_points, model)
    
    # 2. Clarity and coherence
    clarity_score = calculate_clarity_score(summary)
    
    # 3. Grammar (basic check)
    grammar_score = check_grammar_basic(summary)
    
    # 4. Completeness (based on length and NEW coverage score)
    word_count = len(summary.split())
    completeness_score = min(1.0, (semantic_coverage * 0.7 + min(1.0, word_count / 200) * 0.3))
    
    # Weighted combination of scores
    # Semantic Coverage: 55% (Combined 30% Coverage + 25% Similarity)
    # Clarity: 20%
    # Completeness: 15%
    # Grammar: 10%
    final_score = (
        semantic_coverage * 0.55 +
        clarity_score * 0.20 +
        completeness_score * 0.15 +
        grammar_score * 0.10
    )
    
    # Convert to 0-10 scale
    score_0_10 = final_score * 10
    
    # Generate explanation
    explanation = generate_explanation(
        score_0_10, semantic_coverage, clarity_score, 
        completeness_score, grammar_score, summary, word_count
    )
    
    return round(score_0_10, 1), explanation

print("Main evaluation function defined successfully!")

Main evaluation function defined successfully!


In [12]:
def process_all_summaries(student_df, lecture_transcript, key_points, model):
    """
    Process all student summaries and generate scores and explanations.
    
    Args:
        student_df (pd.DataFrame): DataFrame with student information and summaries
        lecture_transcript (str): Full lecture transcript
        key_points (list): List of key points from the lecture
        model: SentenceTransformer model
    
    Returns:
        pd.DataFrame: DataFrame with all original columns plus Score and Explanation
    """
    # Validate inputs
    if student_df is None or len(student_df) == 0:
        raise ValueError("Student DataFrame is empty or None")
    
    if not lecture_transcript or len(lecture_transcript.strip()) < 50:
        raise ValueError("Lecture transcript is empty or too short")
    
    if not key_points or len(key_points) == 0:
        raise ValueError("No key points extracted from lecture transcript")
    
    results = []
    
    # Determine the summary column name (handle variations)
    summary_col = None
    possible_names = ['Summary', 'summary', 'Summary Text', 'Student Summary']
    for name in possible_names:
        if name in student_df.columns:
            summary_col = name
            break
    
    if summary_col is None:
        # Try to find a column that might contain summaries
        for col in student_df.columns:
            if 'summary' in col.lower():
                summary_col = col
                break
    
    if summary_col is None:
        raise ValueError(f"Could not find summary column. Available columns: {list(student_df.columns)}")
    
    print(f"Using column '{summary_col}' for summaries")
    print(f"\nProcessing {len(student_df)} summaries...")
    
    for idx, row in student_df.iterrows():
        # Handle NaN and None values
        summary = str(row[summary_col]) if pd.notna(row[summary_col]) else ""
        summary = summary.strip() if summary else ""
        
        print(f"Processing student {idx + 1}/{len(student_df)}...", end="\r")
        
        try:
            # Evaluate the summary
            score, explanation = evaluate_summary(summary, lecture_transcript, key_points, model)
        except Exception as e:
            print(f"\nWarning: Error evaluating summary for student {idx + 1}: {e}")
            score = 0.0
            explanation = f"Error during evaluation: {str(e)}"
        
        # Create result row - preserve all original columns
        result_row = row.to_dict()
        result_row['Numerical Score'] = score
        result_row['Short explanation'] = explanation
        
        results.append(result_row)
    
    print(f"\n\nCompleted processing all {len(student_df)} summaries!")
    
    # Create results DataFrame
    if not results:
        raise ValueError("No results generated. Please check your input data.")
    
    results_df = pd.DataFrame(results)
    
    return results_df

# Process all summaries
print("Starting batch processing of all student summaries...")
graded_results = process_all_summaries(student_df, lecture_transcript, key_points, model)


Starting batch processing of all student summaries...
Using column 'Summary( upto 100words)' for summaries

Processing 123 summaries...
Processing student 123/123...

Completed processing all 123 summaries!


In [13]:
def prepare_output_dataframe(graded_results):
    """
    Prepare the final output DataFrame with all required columns in the correct order.
    
    Args:
        graded_results (pd.DataFrame): DataFrame with graded results
    
    Returns:
        pd.DataFrame: Formatted output DataFrame
    """
    # Validate that required columns exist
    if 'Numerical Score' not in graded_results.columns:
        raise ValueError("Missing 'Numerical Score' column in graded results")
    if 'Short explanation' not in graded_results.columns:
        raise ValueError("Missing 'Short explanation' column in graded results")
    
    # Map column names (handle variations)
    column_mapping = {
        'Email Address': ['Email Address', 'Email', 'email', 'Email address', 'Email Address'],
        'Name': ['Name of the Student', 'Name', 'Student Name', 'name', 'Name of Student'],
        'Roll Number': ['Roll Number', 'Roll No', 'Roll', 'roll number', 'RollNumber'],
        'Institute': ['Institute', 'institute', 'Institution', 'Institution Name'],
        'Original Summary': None,  # Will be determined from summary column
        'Numerical Score': 'Numerical Score',
        'Short explanation': 'Short explanation'
    }
    
    output_data = {}
    
    # Find and map columns
    for target_col, possible_names in column_mapping.items():
        if possible_names is None:
            # Special handling for Original Summary
            summary_col = None
            for col in graded_results.columns:
                if 'summary' in col.lower() and col not in ['Short explanation', 'Short Explanation']:
                    summary_col = col
                    break
            if summary_col:
                output_data[target_col] = graded_results[summary_col].fillna("")
            else:
                # Create empty column if summary column not found
                output_data[target_col] = [""] * len(graded_results)
        else:
            found = False
            for possible_name in possible_names:
                if possible_name in graded_results.columns:
                    # Fill NaN values with empty string
                    output_data[target_col] = graded_results[possible_name].fillna("")
                    found = True
                    break
            if not found:
                # If column not found, create empty column
                output_data[target_col] = [""] * len(graded_results)
    
    # Ensure all columns have the same length
    expected_length = len(graded_results)
    for key, value in output_data.items():
        if isinstance(value, pd.Series):
            if len(value) != expected_length:
                raise ValueError(f"Column '{key}' has incorrect length: {len(value)} != {expected_length}")
        elif isinstance(value, list):
            if len(value) != expected_length:
                output_data[key] = [""] * expected_length
    
    # Create output DataFrame with required columns in order
    output_df = pd.DataFrame({
        'Email Address': output_data.get('Email Address', [""] * expected_length),
        'Name': output_data.get('Name', [""] * expected_length),
        'Roll Number': output_data.get('Roll Number', [""] * expected_length),
        'Institute': output_data.get('Institute', [""] * expected_length),
        'Original Summary': output_data.get('Original Summary', [""] * expected_length),
        'Numerical Score': graded_results['Numerical Score'],
        'Short explanation': graded_results['Short explanation']
    })
    
    # Validate output DataFrame
    if len(output_df) == 0:
        raise ValueError("Output DataFrame is empty")
    
    return output_df

# Prepare output DataFrame
output_df = prepare_output_dataframe(graded_results)

print("Output DataFrame prepared!")
print(f"\nColumns: {list(output_df.columns)}")
print(f"\nShape: {output_df.shape}")
print(f"\nFirst few rows:")
print(output_df.head())


Output DataFrame prepared!

Columns: ['Email Address', 'Name', 'Roll Number', 'Institute', 'Original Summary', 'Numerical Score', 'Short explanation']

Shape: (123, 7)

First few rows:
                   Email Address            Name Roll Number     Institute  \
0          cs22b1024@iiitr.ac.in  Deepak sharma   Cs22b1024   IIIT Raichur   
1  jyoti.24phdcs08@iiitdwd.ac.in     Jyoti Gadad   24PHDCS08  IIIT Dharwad   
2          cs22b1003@iiitr.ac.in   Aditya Gupta   Cs22b1003   IIIT Raichur   
3          cs22b1007@iiitr.ac.in  Aman Chaurasia   CS22B1007  IIIT Raichur   
4         23bds032@iiitdwd.ac.in  Manikesh Kumar    23bds032  IIIT Dharwad   

                                    Original Summary  Numerical Score  \
0  We studied model selection from a wide range o...              4.6   
1  LangChain helps in building pipelines where an...              8.1   
2  In today’s session, we focused on model select...              6.6   
3  Today, I learned about the implementation of c...  

In [14]:
def export_to_excel(output_df, output_path):
    """
    Export the graded results to an Excel file.
    
    Args:
        output_df (pd.DataFrame): DataFrame with graded results
        output_path (str): Path to output Excel file
    """
    try:
        # Validate DataFrame before export
        if output_df is None or len(output_df) == 0:
            raise ValueError("Cannot export empty DataFrame")
        
        # Check if required columns exist
        required_cols = ['Email Address', 'Name', 'Roll Number', 'Institute', 
                        'Original Summary', 'Numerical Score', 'Short explanation']
        missing_cols = [col for col in required_cols if col not in output_df.columns]
        if missing_cols:
            raise ValueError(f"Missing required columns: {missing_cols}")
        
        # Ensure output directory exists
        output_dir = os.path.dirname(output_path) if os.path.dirname(output_path) else "."
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir, exist_ok=True)
        
        # Export to Excel
        output_df.to_excel(output_path, index=False, engine='openpyxl')
        
        # Verify file was created
        if not os.path.exists(output_path):
            raise FileNotFoundError(f"Output file was not created: {output_path}")
        
        print(f"\nSuccessfully exported results to {output_path}")
        print(f"Total students graded: {len(output_df)}")
        print(f"\nScore statistics:")
        print(f"  Mean: {output_df['Numerical Score'].mean():.2f}")
        print(f"  Min: {output_df['Numerical Score'].min():.2f}")
        print(f"  Max: {output_df['Numerical Score'].max():.2f}")
        print(f"  Std: {output_df['Numerical Score'].std():.2f}")
        
        # Show file size
        file_size = os.path.getsize(output_path) / 1024  # KB
        print(f"  File size: {file_size:.2f} KB")
        
    except Exception as e:
        print(f"Error exporting to Excel: {e}")
        raise

# Export results
export_to_excel(output_df, OUTPUT_FILE_PATH)

print("\n" + "="*50)
print("GRADING COMPLETE!")
print("="*50)
print(f"Output file: {OUTPUT_FILE_PATH}")
print(f"File location: {os.path.abspath(OUTPUT_FILE_PATH)}")



Successfully exported results to Gradedresults_23rd_august.xlsx
Total students graded: 123

Score statistics:
  Mean: 7.03
  Min: 3.60
  Max: 9.80
  Std: 1.16
  File size: 37.50 KB

GRADING COMPLETE!
Output file: Gradedresults_23rd_august.xlsx
File location: c:\Users\jallu\OneDrive\Desktop\Quiz\Gradedresults_23rd_august.xlsx
